In [1]:
import numpy as np
import pandas as pd

# Edit distance example: Building inspection reports

Next we will be looking at a dataset with free-form human-entered text that has a field with differences that are not meaningful.  Meaningless differences between records that should be identical make it more challenging to find parts of the dataset that are similar (and should be treated the same).  

This dataset is from the City of Chicago data portal, and documents building inspections in the 
City of Chicago Department of Buildings.  City of Chicago Data Portal.  https://data.cityofchicago.org/Buildings/Building-Violations/22u3-xenr
Accessed September 20, 2023, filtered for violations from the first half of January 2022.

In [2]:
violations_df = pd.read_csv("../../data/ChicagoBuildingViolationsEarlyJanuary2023.csv")

If your environment does not have the `levenshtein` package installed, see if you can install it. 

In [3]:
%pip install levenshtein
#!conda install -c conda-forge python-levenshtein

In [4]:
violations_df

,ID,VIOLATION LAST MODIFIED DATE,VIOLATION DATE,VIOLATION CODE,VIOLATION STATUS,VIOLATION STATUS DATE,VIOLATION DESCRIPTION,VIOLATION LOCATION,VIOLATION INSPECTOR COMMENTS,VIOLATION ORDINANCE,...,STREET NAME,STREET TYPE,PROPERTY GROUP,SSA,LATITUDE,LONGITUDE,LOCATION,YEAR,MONTH,DAY
0,6729404,06/27/2023,01/10/2022,EV1111,COMPLIED,06/27/2023,MAINTAIN OR REPAIR HYDRO ELEVA,NaN,CITY FIRE RECALL TEST,Failed to maintain hydraulic elevator equipmen...,...,OAKDALE,AVE,8066,8.0,41.935438,-87.642694,"(41.93543799801343, -87.64269400121434)",2022,1,10
1,6726715,09/06/2022,01/03/2022,CN070024,COMPLIED,09/06/2022,REPAIR PORCH SYSTEM,OTHER : :OTHER371335,"FRONT PORCH - ROTTED WOOD CEILING, MISSING ALU...",Failed to repair or replace defective or missi...,...,STATE,ST,535078,NaN,41.675337,-87.622431,"(41.67533708701544, -87.6224312766971)",2022,1,3
2,6726705,06/29/2023,01/03/2022,CN071024,COMPLIED,06/29/2023,REPAIR STOOP,OTHER : :OTHER,"FRONT STOOP, SIDE WALLS - WASHED OUT MORTAR ( ...",Failed to maintain stoop in safe condition and...,...,ESCANABA,AVE,435511,NaN,41.749876,-87.553927,"(41.749876029131826, -87.55392704663606)",2022,1,3
3,6731812,07/14/2023,01/13/2022,CN062024,COMPLIED,07/14/2023,PARAPET,OTHER : :OTHER,SOUTHWEST FRONT GARAGE PERAPET - WASHED OUT MO...,Failed to maintain parapet wall in good repair...,...,103RD,ST,24594,NaN,41.706930,-87.633353,"(41.70693036655445, -87.6333526539513)",2022,1,13
4,6730529,06/13/2022,01/12/2022,CN136026,COMPLIED,06/13/2022,INSECTS,OTHER : :OTHER,BED BUG INFESTATION THROUGHOUT. - PROPERTY MUS...,Exterminate insects and keep dwelling insect-f...,...,LINCOLN,AVE,12595,NaN,41.983469,-87.695449,"(41.98346922607376, -87.69544875484024)",2022,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1827,6730801,06/22/2023,01/12/2022,NC2020,COMPLIED,06/22/2023,PERMIT REQUIRED,OTHER : CANOPY REMOVED PER PERMIT #101003272,REAR YARD-OVERHEAD CANOPY ROOF INSTALLED BETWE...,"Failed to obtain building permit for erecting,...",...,MAJOR,AVE,123856,NaN,41.932068,-87.768851,"(41.93206815540178, -87.76885130741938)",2022,1,12
1828,6730802,06/22/2023,01/12/2022,NC2071,COMPLIED,06/22/2023,WORK NOT ALLOWED,OTHER : :OTHER,"REAR YARD-6-'4"" HIGH VINYL FENCE INSTALLED AT ...",Remove work performed without permit and resto...,...,MAJOR,AVE,123856,NaN,41.932068,-87.768851,"(41.93206815540178, -87.76885130741938)",2022,1,12
1829,6730803,06/22/2023,01/12/2022,NC2020,COMPLIED,06/22/2023,PERMIT REQUIRED,OTHER : :OTHER,REAR YARD-OVERHEAD CANOPY ROOF INSTALLED BETWE...,"Failed to obtain building permit for erecting,...",...,MAJOR,AVE,123856,NaN,41.932068,-87.768851,"(41.93206815540178, -87.76885130741938)",2022,1,12
1830,6735084,06/22/2023,01/04/2022,CN132016,OPEN,NaN,HEAT UNIT ADEQUATELY,NaN,"LIVING ROOM 59 DEGREES, BEDROOM 60 DEGREES. GA...",Heat dwelling unit adequately from September 1...,...,WENTWORTH,AVE,20105,NaN,41.766092,-87.629642,"(41.76609247175032, -87.62964151748226)",2022,1,4


Let's poke around the data just a little:

In [5]:
violations_df["VIOLATION DATE"].value_counts().sort_index()

VIOLATION DATE
01/03/2022    205
01/04/2022    147
01/05/2022    160
01/06/2022    107
01/07/2022    229
01/08/2022    121
01/09/2022     27
01/10/2022    153
01/11/2022    127
01/12/2022    180
01/13/2022    119
01/14/2022    208
01/15/2022     49
Name: count, dtype: int64

These dates are from the first two weeks of 2022.  

Building inspectors don't record many violations on Sundays.  

Let us look at the `VIOLATION DESCRIPTION` field:

In [6]:
violations_df["VIOLATION DESCRIPTION"].value_counts()

VIOLATION DESCRIPTION
MAINTAIN OR REPAIR ELECT ELEVA    222
MAINTAIN OR REPAIR HYDRO ELEVA    123
HEAT UNIT ADEQUATELY               89
SAFE WORKING CONDITIONS            64
ARRANGE PREMISE INSPECTION         57
                                 ... 
FIRE EXTNGSHR REQ, NONRESDNTL       1
ARRANGE FOR INSPECTION              1
ACCESS TO ROOF                      1
CEILING HEIGHT - 7'                 1
NO CLOSET/STORAGE UNDER STAIR       1
Name: count, Length: 233, dtype: int64

In [7]:
violations_df["VIOLATION INSPECTOR COMMENTS"].value_counts()

VIOLATION INSPECTOR COMMENTS
INSPECTION                                                                                                                                                                                                                                                             7
STOP ALL WORK ON PREMISES UNTIL APPROVED PLANS AND BUILDING PERMIT ARE OBTAINED 13-12-080 13-32-035                                                                                                                                                                    5
SCHEDULE ELEVATOR INSPECTION WITH CITY OF CHICAGO ELEVATOR BUREAU                                                                                                                                                                                                      4
1ST FLOOR/RESIDENTIAL ENTRY DOOR - GAPS AT BOTTOM OF DOOR ALLOWING RATS TO ENTER BUILDING                                                                                       

Ok, so 1580 different comments for 1832 rows.  Are any values missing?

In [8]:
len(violations_df), violations_df["VIOLATION INSPECTOR COMMENTS"].isnull().sum()

(1832, 98)

Just counting, we have 1832 records, 98 of which (5.3%) are missing inspector comments.
Of the remaining 1734 records with comments, there are 1579 different comments.
This means there are 253 more records than distinct comments, so 13.8% of the 
comments are exact duplicates somewhere within the dataset.  

Some pairs of records, however, have non-identical comments that probably should 
be identical.  We can use the Levenshtein distance to identify similar but not exactly 
identical comments; we will make some decisions about how to merge records, and produce a 
dataset that has had the text of similar comments merged.

## All-against all comment comparison

First, we will compare all the distinct comments to all the other comments.  
(This requires $n^2$ comparisons, where n is the 1832 distinct comments
in the `VIOLLATION INSPECTOR COMMENTS` field.)  We can afford 3.3 million comparisons between strings.    

In [9]:
import Levenshtein
from Levenshtein import distance

In [10]:
distinctcomments = violations_df["VIOLATION INSPECTOR COMMENTS"].value_counts().index
distinctcommentnumber = violations_df["VIOLATION INSPECTOR COMMENTS"].value_counts().values

for i in range(len(distinctcomments)):
    for j in range(len(distinctcomments)):
        d = distance(distinctcomments[i], distinctcomments[j])
        # Find all the pairs that have Levenshtein distance 1 
        if d == 1:  
            # And print out the number of times and each of the pair.
            print(distinctcommentnumber[i], distinctcomments[i])
            print(distinctcommentnumber[j], distinctcomments[j])


5 STOP ALL WORK ON PREMISES UNTIL APPROVED PLANS AND BUILDING PERMIT ARE OBTAINED 13-12-080 13-32-035
1 STOP ALL WORK ON PREMISES UNTIL APPROVED PLANS AND BUILDING PERMIT AE OBTAINED 13-12-080 13-32-035
4 CHIMNEY - MISSING CAP
1 CHIMNEY -  MISSING CAP
4 USING COOKING STOVE AS HEATING DEVICE.
1 USING COOKING STOVE AS HEATING DEVICE
3 WORKING DOOR RESTRICTOR REQUIRED.
1 WORKING DOOR RESTRICTOR REQIRED.
2 STOVE HEAT/SPACE HEATERS.
1 STOVE HEAT/SPACE HEATER.
2 EMPLOY LICENSED AND BONDED PLUMBING CONTRACTOR FOR PLUMBING WORK TO BE PERFORMED TO ADDRESS PLUMBING VIOLATIONS
1 EMPLOY LICENSED AND BONDED PLUMBING CONTRACTOR FOR PLUMBING WORK TO BE PERFORMED TO ADDRESS PLUMBING VIOLATIONS.
2 MISSING SMOKE DETECTORS.
1 MISSING SMOKE DETECTOR.
2 AT BASEMENT
1 AT BASEMENT.
2 A MAINTENANCE CONTROL PROGRAM IS REQUIRED TO BE ON SITE.
1 AMAINTENANCE CONTROL PROGRAM IS REQUIRED TO BE ON SITE.
1 PROVIDE ELECTRICAL PERMIT IN ORDER TO CORRECT ALL ELECTRICAL VIOLATIONS  (14A-4-404.6)
1 PROVIDE ELECTRICAL PER

Note:  This all-against all comparison computes both

$$ d(\textrm{comment}_i, \textrm{comment}_j) $$

and

$$ d(\textrm{comment}_j, \textrm{comment}_i) $$

which are always equal.  So this double-counts all the pairs.  Note also we aren't storing the distances, just calculating all of them and taking action for some values of the distance.

Glancing at these pairs of comments, somme are misspellings (UNVERFIED, CAT!, AE) and some are differences in 
punctuation.   Surely, `CHIMNEY - MISSING CAP` is not meaningfully different from `CHIMNEY -  MISSING CAP` 
with two spaces after the hyphen.  These are meaningless differences.

A handful of the differences, however, may be meaningful.  CAT1 and CAT5 are different inspection schedules 
for different types of elevators, and the two comments have a distance of only 1.  Differences in addresses, 
directions, permit numbers and cardinal directions can have small edit distances but can change the meaning.

Creating a new field that merges comments that are more similar than a certain edit distance risks corrupting some
of the data (SOUTH and NORTH have edit distance 2).  We should always preserve the original data when 
"cleaning" in this way.   But merging comments that mean the same thing will make it easier to find groups and 
understand the data by removing differences (in punctuation and spelling) that are in no way relevant to the 
inspections.

To merge the similar-but-not-identical comments, we will choose a threshold for merging comments, 
create a dictionary that maps old comments to new cleaned up comments, write a function that uses 
this dictionary to replace the comments that are on the list, apply this function, and finally check
that we get what we expect.

First, let us decide on a threshold.  What do comments look like at d=4?

In [11]:
for i in range(len(distinctcomments)):
    for j in range(len(distinctcomments)):
        d = distance(distinctcomments[i], distinctcomments[j])
        if d == 4:   # Find all the pairs that have Levenshtein distance 1 
            print(distinctcommentnumber[i], distinctcomments[i])
            print(distinctcommentnumber[j], distinctcomments[j])

2 INTERIOR OF BUILDING - ROACH INFESTATION
2 INTERIOR OF BUILDING - MICE INFESTATION
2 INTERIOR OF BUILDING - MICE INFESTATION
2 INTERIOR OF BUILDING - ROACH INFESTATION
2 STOP WORK UNTIL PERMIT IS OBTAINED SECTION 14A-4-411
1 STOP WORK UNTIL NEW PERMIT IS OBTAINED SECTION 14A-4-411
1 CHIMNEY - FRACTURES WITH LOOSE MORTAR
1 CHIMNEY - FRACTURES AND LOOSE MORTAR
1 CHIMNEY - FRACTURES AND LOOSE MORTAR
1 CHIMNEY - FRACTURES WITH LOOSE MORTAR
1 CHIMNEY - FRACTURES AND LOOSE MORTAR
1 CHIMNEY -  FRACTURES , LOOSE MORTAR
1 CHIMNEY - FRACTURES AND LOOSE MORTAR
1 CHIMNEY - FRACTURES ,LOOSE MORTAR
1 NORTH CORNICE - WASHED OUT MORTAR.
1 WEST CORNICE - WASHED OUT MORTAR.
1 WEST CORNICE - WASHED OUT MORTAR.
1 NORTH CORNICE - WASHED OUT MORTAR.
1 CHIMNEY -  FRACTURES , LOOSE MORTAR
1 CHIMNEY - FRACTURES AND LOOSE MORTAR
1 STOP WORK UNTIL NEW PERMIT IS OBTAINED SECTION 14A-4-411
2 STOP WORK UNTIL PERMIT IS OBTAINED SECTION 14A-4-411
1 CHIMNEY - FRACTURES ,LOOSE MORTAR
1 CHIMNEY - FRACTURES AND LOOSE M

These all look pretty solidly identical.  The insertion of the word "NEW", the replacement of " ," with " AND ".  We can merge these.

What about d=6?

In [12]:
for i in range(len(distinctcomments)):
    for j in range(len(distinctcomments)):
        d = distance(distinctcomments[i], distinctcomments[j])
        if d == 6:   # Find all the pairs that have Levenshtein distance 1 
            print(distinctcommentnumber[i], distinctcomments[i])
            print(distinctcommentnumber[j], distinctcomments[j])

3 WEST, EAST, NORTH, ELEVATIONS - ABANDONED VEHICLE IN REAR YARD, BROKEN AWNING ON FRONT PORCH; LOTS OF JUNK, GARBAGE, APPLIANCES, BAGS OF CONCRETE, SHOPPING CARTS, AUTO PARTS AND TIRES, TELEVISION, BROKEN DOORS, TRASH, DEBRIS, ETC. THROUGHOUT; EXCESSIVE C
2 WEST, EAST, NORTH, ELEVATIONS - ABANDONED VEHICLE IN REAR YARD, BROKEN AWNING ON FRONT PORCH; LOTS OF JUNK, GARBAGE, APPLIANCES, BAGS OF CONCRETE, SHOPPING CARTS, AUTO PARTS AND TIRES, TELEVISION, BROKEN DOORS, TRASH, DEBRIS, ETC. THROUGHOUT; EXCESSIVE CLUTTER
2 NO SMOKE DETECTORS.
2 MISSING SMOKE DETECTORS.
2 WEST, EAST, NORTH, ELEVATIONS - ABANDONED VEHICLE IN REAR YARD, BROKEN AWNING ON FRONT PORCH; LOTS OF JUNK, GARBAGE, APPLIANCES, BAGS OF CONCRETE, SHOPPING CARTS, AUTO PARTS AND TIRES, TELEVISION, BROKEN DOORS, TRASH, DEBRIS, ETC. THROUGHOUT; EXCESSIVE CLUTTER
3 WEST, EAST, NORTH, ELEVATIONS - ABANDONED VEHICLE IN REAR YARD, BROKEN AWNING ON FRONT PORCH; LOTS OF JUNK, GARBAGE, APPLIANCES, BAGS OF CONCRETE, SHOPPING CARTS, AUT

This is clearly too much.  `NO HEAT` should not be merged with `PANELS` or `FIRE TEST`.   

What about d=5?

In [13]:
for i in range(len(distinctcomments)):
    for j in range(len(distinctcomments)):
        d = distance(distinctcomments[i], distinctcomments[j])
        if d == 5:   # Find all the pairs that have Levenshtein distance 1 
            print(distinctcommentnumber[i], distinctcomments[i])
            print(distinctcommentnumber[j], distinctcomments[j])

2 SOUTH ELEVATION / DOWNSPOUT - MISSING.
2 WEST ELEVATION / DOWNSPOUT - MISSING
2 REAR EXTERIOR STAIRS,GRADE TO BASEMENT-MISSING HAND RAIL.
2 FRONT EXTERIOR STAIRS-GRADE TO BASEMENT-MISSING HAND RAIL.
2 FRONT EXTERIOR STAIRS-GRADE TO BASEMENT-MISSING HAND RAIL.
2 REAR EXTERIOR STAIRS,GRADE TO BASEMENT-MISSING HAND RAIL.
2 CODE COMPLIANT PIT LADDER REQUIRED.
1 A CODE COMPLIANT PIT LADDER IS REQUIRED.
2 WEST ELEVATION / DOWNSPOUT - MISSING
2 SOUTH ELEVATION / DOWNSPOUT - MISSING.
1 CHIMNEY - FRACTURES WITH LOOSE MORTAR
1 CHIMNEY -  FRACTURES , LOOSE MORTAR
1 CHIMNEY - FRACTURES WITH LOOSE MORTAR
1 CHIMNEY - FRACTURES ,LOOSE MORTAR
1 BUILDING INFESTED WITH MICE, INCLUDING APT. 2E.
1 BUILDING INFESTED WITH ROACHES, INCLUDING APT. 2E.
1 BUILDING INFESTED WITH ROACHES, INCLUDING APT. 2E.
1 BUILDING INFESTED WITH MICE, INCLUDING APT. 2E.
1 EAST PASSENGER: REPAIR EMERGENCY PHONE
1 EAST PASSENGER: REPAIR EMERGENCY ALARM
1 SOUTH ELEVATION / COPING - OPEN MORTAR JOINTS.
1 SOUTH ELEVATION / CORNIC

These all look similar enough to merge.  Let us merge comments with d less than or equal 5.

## Apply the changes

In [14]:
renamedict = {}

for i in range(len(distinctcomments)):
    for j in range(len(distinctcomments)):
        d = distance(distinctcomments[i], distinctcomments[j])
        # Find all the pairs that have Levenshtein distance  <=5 
        if d <= 5 and d !=0  :  
            # If comment i is used fewer times than j   
            if distinctcommentnumber[i] < distinctcommentnumber[j] :  
            # Flag comment i to be changed to comment j 
                 renamedict[distinctcomments[i]] = distinctcomments[j]
renamedict, len(renamedict)

({'MISSING SMOKE DETECTOR.': 'MISSING SMOKE DETECTORS.',
  'EMPLOY LICENSED AND BONDED PLUMBING CONTRACTOR FOR PLUMBING WORK TO BE PERFORMED TO ADDRESS PLUMBING VIOLATIONS.': 'EMPLOY LICENSED AND BONDED PLUMBING CONTRACTOR FOR PLUMBING WORK TO BE PERFORMED TO ADDRESS PLUMBING VIOLATIONS',
  'STOP WORK UNTIL A PERMIT IS OBTAINED SECTION 14A-4-411': 'STOP WORK UNTIL PERMIT IS OBTAINED SECTION 14A-4-411',
  'WORKING DOOR RESTRICTOR REQIRED.': 'WORKING DOOR RESTRICTOR REQUIRED.',
  'STOP ALL WORK ON PREMISES UNTIL APPROVED PLANS AND BUILDING PERMIT AE OBTAINED 13-12-080 13-32-035': 'STOP ALL WORK ON PREMISES UNTIL APPROVED PLANS AND BUILDING PERMIT ARE OBTAINED 13-12-080 13-32-035',
  'STOP WORK UNTIL NEW PERMIT IS OBTAINED SECTION 14A-4-411': 'STOP WORK UNTIL PERMIT IS OBTAINED SECTION 14A-4-411',
  'STOVE HEAT/SPACE HEATER.': 'STOVE HEAT/SPACE HEATERS.',
  'CHIMNEY -  MISSING CAP': 'CHIMNEY - MISSING CAP',
  'AT BASEMENT.': 'AT BASEMENT',
  'USING COOKING STOVE AS HEATING DEVICE': 'USING

This looks reasonable.  Write the function: 

In [15]:
def renamecomment(comment):
    if comment in renamedict.keys():
        return renamedict[comment]
    else:
        return comment

I don't want to write a function without some kind of testing to see that it does what I think it does.
I'll write two tests, simple invocations of `renamecomment` with known correct answers.

In [16]:
renamecomment("NOT IN DICTIONARY") == "NOT IN DICTIONARY"

True

In [17]:
renamecomment('MISSING SMOKE DETECTOR') == 'MISSING SMOKE DETECTORS.'

True

Ok, seems to work.  Let us create a new column with `apply`: 

In [18]:
violations_df["COMMENTS FIXED"] = violations_df["VIOLATION INSPECTOR COMMENTS"].apply(renamecomment)

And how do we know that this did anything?  Let's check which rows were changed:  

In [19]:
violations_df.query("`COMMENTS FIXED` != `VIOLATION INSPECTOR COMMENTS`")

,ID,VIOLATION LAST MODIFIED DATE,VIOLATION DATE,VIOLATION CODE,VIOLATION STATUS,VIOLATION STATUS DATE,VIOLATION DESCRIPTION,VIOLATION LOCATION,VIOLATION INSPECTOR COMMENTS,VIOLATION ORDINANCE,...,STREET TYPE,PROPERTY GROUP,SSA,LATITUDE,LONGITUDE,LOCATION,YEAR,MONTH,DAY,COMMENTS FIXED
5,6726415,03/08/2022,01/03/2022,VT1010,COMPLIED,03/08/2022,ARRANGE COMPLETED PERMIT INSP.,NaN,NaN,Arrange mechanical ventilation or warm air hea...,...,AVE,198348,22.0,41.980342,-87.669580,"(41.98034244999325, -87.66958046014629)",2022,1,3,NaN
22,6732325,09/21/2022,01/11/2022,VT1030,COMPLIED,09/20/2022,ARRANGE PERMIT INSP.,1115 N CHRISTIANA AV,NaN,Arrange in writing for mechanical ventilating ...,...,AVE,51247,NaN,41.901508,-87.710433,"(41.901507612598834, -87.71043339750256)",2022,1,11,NaN
23,6735146,03/11/2022,01/09/2022,CN046013,COMPLIED,03/11/2022,UNAPPROVED HEATING DEVICE,NaN,STOVE HEAT/SPACE HEATER.,Stop using cooking or water heating device as ...,...,AVE,21765,NaN,41.691177,-87.612259,"(41.691177471896076, -87.61225906980805)",2022,1,9,STOVE HEAT/SPACE HEATERS.
24,6727334,01/04/2022,01/04/2022,PL151137,OPEN,NaN,OPEN,NaN,NaN,NaN,...,AVE,400614,14.0,41.770689,-87.685700,"(41.77068909096362, -87.68569998583654)",2022,1,4,NaN
25,6730399,01/12/2022,01/11/2022,PLD2290,OPEN,NaN,PROVIDE AUTOMATIC MIXING DEVIC,NaN,NaN,Provide automatic mixing device for public lav...,...,ST,1596,NaN,41.887222,-87.757826,"(41.887221537144235, -87.75782634562349)",2022,1,11,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1598,6729116,08/31/2022,01/10/2022,VT1010,COMPLIED,08/31/2022,ARRANGE COMPLETED PERMIT INSP.,NaN,NaN,Arrange mechanical ventilation or warm air hea...,...,AVE,156823,NaN,41.947644,-87.781798,"(41.947644039902265, -87.78179844629967)",2022,1,10,NaN
1618,6730385,11/21/2022,01/11/2022,VT1010,COMPLIED,11/18/2022,ARRANGE COMPLETED PERMIT INSP.,NaN,NaN,Arrange mechanical ventilation or warm air hea...,...,ST,286211,NaN,41.841823,-87.701135,"(41.84182295978573, -87.70113477596593)",2022,1,11,NaN
1621,6726984,12/07/2022,01/03/2022,PL151137,COMPLIED,12/07/2022,OPEN,NaN,NaN,NaN,...,AVE,60561,NaN,41.904908,-87.676385,"(41.904907565709664, -87.67638534234142)",2022,1,3,NaN
1627,6728833,11/15/2022,01/07/2022,VT1010,OPEN,NaN,ARRANGE COMPLETED PERMIT INSP.,NaN,NaN,Arrange mechanical ventilation or warm air hea...,...,AVE,117688,NaN,41.933936,-87.715618,"(41.933936312229164, -87.71561838752238)",2022,1,7,NaN


112 rows were changed.  Hmm. So what did this do to the number of unique comments?  We expect something like 100 fewer unique comments.. right?

In [20]:
print("Records:               ", len(violations_df))
print("Blank records:         ", violations_df["VIOLATION INSPECTOR COMMENTS"].isnull().sum())
print("Distinct records:      ", len(violations_df["VIOLATION INSPECTOR COMMENTS"].value_counts()) ) 
print("Distinct records fixed:", len(violations_df["COMMENTS FIXED"].value_counts()))

Records:                1832
Blank records:          98
Distinct records:       1580
Distinct records fixed: 1566


This is a little puzzling.  The renaming changed 112 rows but only reduced the number of unique entries by 14.  
This is not what I was expecting; something is not working quite as it should.  Let's look at some of the 112
rows that were changed more closely:

In [21]:
changed = violations_df.query("`COMMENTS FIXED` != `VIOLATION INSPECTOR COMMENTS`") 
changed[["VIOLATION INSPECTOR COMMENTS", "COMMENTS FIXED"]] 

,VIOLATION INSPECTOR COMMENTS,COMMENTS FIXED
5,NaN,NaN
22,NaN,NaN
23,STOVE HEAT/SPACE HEATER.,STOVE HEAT/SPACE HEATERS.
24,NaN,NaN
25,NaN,NaN
...,...,...
1598,NaN,NaN
1618,NaN,NaN
1621,NaN,NaN
1627,NaN,NaN


This is disappointing.  The 112 includes NaNs, since NaN does not match with NaN.  Show only the non-NaN rows: 

In [22]:
changed[~changed["COMMENTS FIXED"].isnull()][["VIOLATION INSPECTOR COMMENTS", "COMMENTS FIXED"]] 

,VIOLATION INSPECTOR COMMENTS,COMMENTS FIXED
23,STOVE HEAT/SPACE HEATER.,STOVE HEAT/SPACE HEATERS.
34,STOP WORK UNTIL NEW PERMIT IS OBTAINED SECTION...,STOP WORK UNTIL PERMIT IS OBTAINED SECTION 14A...
168,AT BASEMENT.,AT BASEMENT
204,CHIMNEY - MISSING CAP,CHIMNEY - MISSING CAP
529,AMAINTENANCE CONTROL PROGRAM IS REQUIRED TO BE...,A MAINTENANCE CONTROL PROGRAM IS REQUIRED TO B...
777,USING COOKING STOVE AS HEATING DEVICE,USING COOKING STOVE AS HEATING DEVICE.
802,MISSING SMOKE DETECTOR,MISSING SMOKE DETECTORS.
876,A CODE COMPLIANT PIT LADDER IS REQUIRED.,CODE COMPLIANT PIT LADDER REQUIRED.
898,CHIMNEYS - MISSING CAPS,CHIMNEY - MISSING CAP
1104,STOP WORK UNTIL A PERMIT IS OBTAINED SECTION 1...,STOP WORK UNTIL PERMIT IS OBTAINED SECTION 14A...


These are the 14 that were changed.  

## Fix doesn't fix enough comments
Something doesn't quite add up here.   Let me count the number of pairs
that were found that were close enough to be merged.

In [23]:
renamedict = {}
n=0
for i in range(len(distinctcomments)):
    for j in range(len(distinctcomments)):
        d = distance(distinctcomments[i], distinctcomments[j])
        # Find all the pairs that have Levenshtein distance  <=5 
        if d <= 5 and d !=0  :
            n = n+1
            # If comment i is used fewer times than j   
            if distinctcommentnumber[i] < distinctcommentnumber[j] :  
            # Flag comment i to be changed to comment j 
                 renamedict[distinctcomments[i]] = distinctcomments[j]
renamedict, len(renamedict), n

({'MISSING SMOKE DETECTOR.': 'MISSING SMOKE DETECTORS.',
  'EMPLOY LICENSED AND BONDED PLUMBING CONTRACTOR FOR PLUMBING WORK TO BE PERFORMED TO ADDRESS PLUMBING VIOLATIONS.': 'EMPLOY LICENSED AND BONDED PLUMBING CONTRACTOR FOR PLUMBING WORK TO BE PERFORMED TO ADDRESS PLUMBING VIOLATIONS',
  'STOP WORK UNTIL A PERMIT IS OBTAINED SECTION 14A-4-411': 'STOP WORK UNTIL PERMIT IS OBTAINED SECTION 14A-4-411',
  'WORKING DOOR RESTRICTOR REQIRED.': 'WORKING DOOR RESTRICTOR REQUIRED.',
  'STOP ALL WORK ON PREMISES UNTIL APPROVED PLANS AND BUILDING PERMIT AE OBTAINED 13-12-080 13-32-035': 'STOP ALL WORK ON PREMISES UNTIL APPROVED PLANS AND BUILDING PERMIT ARE OBTAINED 13-12-080 13-32-035',
  'STOP WORK UNTIL NEW PERMIT IS OBTAINED SECTION 14A-4-411': 'STOP WORK UNTIL PERMIT IS OBTAINED SECTION 14A-4-411',
  'STOVE HEAT/SPACE HEATER.': 'STOVE HEAT/SPACE HEATERS.',
  'CHIMNEY -  MISSING CAP': 'CHIMNEY - MISSING CAP',
  'AT BASEMENT.': 'AT BASEMENT',
  'USING COOKING STOVE AS HEATING DEVICE': 'USING

There were 116 pairs of comments (out of 3.3 million) that were closer than d = 5 but the loop only left
instructions to rename 14 comments?  I need to look closely at the logic here:

Let's talk through the logic here.

Which comments get renamed?  Comments that are more than 0 but 5 or fewer edits away from another comment, and
for which the other comment has strictly greater abundance.  

Aha.  For this dataset, most 
of the comments are unique, and most of the pairs of similar comments have both comments unique.
Comments that could be merged, but that are present only once are ignored.  

Let us add logic that
merges comments even if they have the same number of occurrences.  We need an arbitrary rule to 
pick one comment when the counts are tied:  let us choose the one that is first in alphabetical 
order.  This is a canonicalization choice: we introduce an arbitrary choice between two similar
comments to allow consolidation without having to make a choice that requires human input, as 
choosing the correct spelling would. 

In [24]:
renamedict = {}
n=0
for i in range(len(distinctcomments)):
    for j in range(len(distinctcomments)):
        d = distance(distinctcomments[i], distinctcomments[j])
        # Find all the pairs that have Levenshtein distance  <=5 
        if d <= 5 and d !=0  :
            n = n+1
            # If comment i is used fewer times than j   
            if distinctcommentnumber[i] < distinctcommentnumber[j] :  
            # Flag comment i to be changed to comment j 
                 renamedict[distinctcomments[i]] = distinctcomments[j]
            # If two comments have the same number of counts:
            if distinctcommentnumber[i] == distinctcommentnumber[j]:
                # Rename if j comes first in lexicographic (alphabetical) order:
                if distinctcomments[j] <  distinctcomments[i]:
                    renamedict[distinctcomments[i]] = distinctcomments[j]
renamedict, len(renamedict), n

({'REAR EXTERIOR STAIRS,GRADE TO BASEMENT-MISSING HAND RAIL.': 'FRONT EXTERIOR STAIRS-GRADE TO BASEMENT-MISSING HAND RAIL.',
  'INTERIOR OF BUILDING - ROACH INFESTATION': 'INTERIOR OF BUILDING - MICE INFESTATION',
  'WEST ELEVATION / DOWNSPOUT - MISSING': 'SOUTH ELEVATION / DOWNSPOUT - MISSING.',
  'MISSING CARBON MONOXIDE DETECTOR.': 'MISSING CARBOB MONOXIDE DETECTOR',
  'MISSING SMOKE DETECTOR.': 'MISSING SMOKE DETECTOR',
  'EMPLOY LICENSED AND BONDED PLUMBING CONTRACTOR FOR PLUMBING WORK TO BE PERFORMED TO ADDRESS PLUMBING VIOLATIONS.': 'EMPLOY LICENSED AND BONDED PLUMBING CONTRACTOR FOR PLUMBING WORK TO BE PERFORMED TO ADDRESS PLUMBING VIOLATIONS',
  'CHIMNEY - FRACTURES WITH LOOSE MORTAR': 'CHIMNEY - FRACTURES ,LOOSE MORTAR',
  'CHIMNEY - FRACTURES AND LOOSE MORTAR': 'CHIMNEY - FRACTURES ,LOOSE MORTAR',
  'INTERIOR OF BUILDING /NO RESPONSE. UNVERIFIED DETECTORS AND CONDITIONS. UNVERIFIED OCCUPANCY.': 'INTERIOR OF BUILDING /NO RESPONSE. UNVERIFIED DETECTORS AND CONDITIONS. UNVERFIE

This is more believable.  The 116 pairs should represent 56 real pairs of comments, and some comments might have more than one partner, so there are only 51 renaming rules.

Let us apply this (larger!) comment-renaming fix, and see if it puts a larger dent in the number of unique comments:

In [25]:
violations_df["COMMENTS FIXED2"] = violations_df["VIOLATION INSPECTOR COMMENTS"].apply(renamecomment)

In [26]:
print("Records:               ", len(violations_df))
print("Distinct records:      ", len(violations_df["VIOLATION INSPECTOR COMMENTS"].value_counts()) ) 
print("Distinct records fixed:", len(violations_df["COMMENTS FIXED2"].value_counts()))

Records:                1832
Distinct records:       1580
Distinct records fixed: 1533


Now, although there were 51 comments slated to be renamed, it only reduced the number of unique comments
from 1580 to 1544 (by 47).  The comments are still 5.3% not-a-number, but now 11.0% have non-unique comments.

So we were able to "clean up" 54 out of 1832 (2.9%) of the comment records.  The effect on the unique comments 
is a bit more dramatic than the effect on distinct comments.  Unique comments dropped from 

In [27]:
print ("Unique comments before", violations_df["VIOLATION INSPECTOR COMMENTS"].value_counts().value_counts()[1])
print ("Unique comments after ", violations_df["COMMENTS FIXED2"].value_counts().value_counts()[1])
print ("Out of                 ", len(violations_df))

Unique comments before 1451
Unique comments after  1375
Out of                 1832


The renaming-cleaning procedure reduced the fraction of the data in unique-comment categories by 4%.  These are records 
that can presumably be better matched with other rows with similar content.

## Clean data

This exercise tried to clean up (meaningless) typographic differences in comment fields; in real-world problems, you are likely to encounter two databases with sometimes-matching and sometimes-mismatching data on more than one field.  You might have a database of people from the HR department and another from the directory, or might have a database of addresses from a municipal database and a commercial database.  

Some fields match more reliably than others; if your job is to somehow retrieve the most likely N matches to a test query, your solution will likely try to match on high-quality identifiers first (national ID numbers for people, FIPS codes for places, transaction ID numbers where appropriate) and attempt fuzzy matching only after exhausting the pool of more-likely-correct hits on identifiers, using a score that is a mixture of fuzzy-matching goodness-of-match scores from different fields. 